In [ ]:
import numpy as np
from scipy.spatial import KDTree
from tqdm import tqdm, trange
import sys, os
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import clear_output

from multiprocessing import Pool

from jmutils import *

## Distance between last covered point and the seed which covers it

To test the conjecture about whether one fast-growing ball covers the whole space at the coverage time, we'll measure the average distance between the last pixel to be covered and the seed which covers it (and maybe whether that seed also covers the whole space at the coverage time, which is pretty easy to check).

Since we're not drawing a picture, there's no real reason to restrict ourselves to $d=2$ here; we could do it for $d=3$ and even $d \geq 4$. Moulinec's algorithm still works perfectly well, although computing the points in each ball may be slow.

Could I speed things up by just checking the last point _on the boundary_ to be covered?

In [ ]:
# Reset the configuration
n = 1000000
dimension = 3
resolution = 100
exponents = np.linspace(0.5,3.5,31,endpoint=True)
distances = np.zeros_like(exponents)
count = 0

In [ ]:
# "Meta-configuration" not affecting the simulation results.
repetitions = 1000
PARALLEL = 2

In [ ]:
# Run some simulations.
max_time = 2*( np.log(n) / (np.pi * n) )**(1/dimension)

def get_distance(a):
    rates = U**(-1/a)
    overtaken = get_overtake_times(rates,dists,fastest_index)
    assignments, times = assign_cells_random_radii(seeds,rates,overtaken,resolution,max_time)
    last = np.unravel_index(np.argmax(times),times.shape)
    last_location = np.array( [last[0] / (resolution-1), last[1] / (resolution-1)] )
    covered_by = assignments[last]
    return np.linalg.norm(last_location - seeds[covered_by])

def get_distance_3d(a):
    rates = U**(-1/a)
    overtaken = get_overtake_times(rates,dists,fastest_index)
    assignments, times = assign_cells_random_radii_3d(seeds,rates,overtaken,resolution,max_time)
    last = np.unravel_index(np.argmax(times),times.shape)
    last_location = np.array( [last[0] / (resolution-1), last[1] / (resolution-1), last[2] / (resolution-1)] )
    covered_by = assignments[last]
    return np.linalg.norm(last_location - seeds[covered_by])

def live_plot(data):
    clear_output(wait=True)
    fig,ax = plt.subplots()
    ax.plot(exponents,data)
    ax.set_ylim(0,np.sqrt(dimension))
    plt.show()

for k in trange(repetitions):
    seeds = sample_points(n,d=dimension)
    U = np.random.random(size=seeds.shape[0]) # Random heavy-tailed radii
    fastest_index = np.argmin(U)
    fastest_seed = seeds[fastest_index]
    dists = np.linalg.norm(seeds - fastest_seed, axis=1)
    if dimension == 2:
        with Pool(PARALLEL) as p:
            new_distance = p.map(get_distance, exponents)
    elif dimension == 3:
        with Pool(PARALLEL) as p:
            new_distance = p.map(get_distance_3d, exponents)
    else:
        break
    distances += new_distance
    count += 1
    live_plot(distances/count)

In [ ]:
fig, ax = plt.subplots()
avg_distances = distances / count
ax.plot(exponents,avg_distances)
ax.set_title(f'n={n}, d={dimension}')
ax.set_xlabel("$\\alpha$")
ax.set_ylabel("average distance")
plt.savefig(f'avg-distance-d{dimension}-n{n}.pdf')
plt.show()